In [278]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sqlite3 import connect

In [279]:
shooting = pd.read_csv("data/lmpd_shootings.csv")
lmpd = pd.read_csv("data/ot_added_all.csv")

In [280]:
shooting.head()

,year,Name,Race,Sex,Age,Ethnicity,Years of Service,lethal
0,2021,Reymart Relos,H,M,30,U,4 months,N
1,2021,Jonathan Haywood,B,M,45,U,7,N
2,2021,Michael Crowe,W,M,28,U,3,N
3,2021,Michael Conner,W,M,37,U,9,N
4,2021,Galen Hinshaw,W,M,32,U,6,n


In [281]:
# lmpd['Department'].unique()

In [282]:
lmpd = lmpd[(lmpd['Department'] == "Louisville Metro Police Department") | (lmpd['Department'] == "Louisville Metro Police")]
lmpd.head()

,Unnamed: 0,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Hr_Worked
0,9291,2023,"Ingram, Lydia Ann",Louisville Metro Police Department,LMPD Service Center Technician,41995.2,0.00,151.43,0.00,0.0,151.43,9292,0.000000,0.000000,inf
53,28134,2020,"Patterson, William",Louisville Metro Police,Police Sergeant,71448.0,9691.28,14040.60,4240.80,11721.6,39694.28,28135,4.659269,6.988904,78.634319
71,8164,2023,"Williams, Derrick D",Louisville Metro Police Department,Police Officer,75462.4,50270.10,69444.05,7685.63,715.0,128114.78,8165,24.168317,36.252476,76.837829
85,9084,2023,"Mattiche, Youssef",Louisville Metro Police Department,Police Officer,72862.4,50943.72,68303.79,6786.84,130.0,126164.35,9085,24.492173,36.738260,75.753855
105,8446,2023,"Jenkins, Todd Barry",Louisville Metro Police Department,Police Sergeant,92872.0,64424.36,84087.13,6899.98,65.0,155476.47,8447,30.973250,46.459875,74.805522


In [283]:
lmpd['Employee_Name'] = lmpd['Employee_Name'].str.split(', ').str[::-1].str.join(' ')
lmpd['Employee_Name'] = lmpd['Employee_Name'].str.split().str[0] + ' ' + lmpd['Employee_Name'].str.split().str[-1]

# lmpd['Employee_Name'] = lmpd['Employee_Name'].str.replace(r'\b[A-Z]\.\s*', '', regex=True)

lmpd.head()

,Unnamed: 0,CalYear,Employee_Name,Department,jobTitle,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Hr_Worked
0,9291,2023,Lydia Ingram,Louisville Metro Police Department,LMPD Service Center Technician,41995.2,0.00,151.43,0.00,0.0,151.43,9292,0.000000,0.000000,inf
53,28134,2020,William Patterson,Louisville Metro Police,Police Sergeant,71448.0,9691.28,14040.60,4240.80,11721.6,39694.28,28135,4.659269,6.988904,78.634319
71,8164,2023,Derrick Williams,Louisville Metro Police Department,Police Officer,75462.4,50270.10,69444.05,7685.63,715.0,128114.78,8165,24.168317,36.252476,76.837829
85,9084,2023,Youssef Mattiche,Louisville Metro Police Department,Police Officer,72862.4,50943.72,68303.79,6786.84,130.0,126164.35,9085,24.492173,36.738260,75.753855
105,8446,2023,Todd Jenkins,Louisville Metro Police Department,Police Sergeant,92872.0,64424.36,84087.13,6899.98,65.0,155476.47,8447,30.973250,46.459875,74.805522


In [284]:
# lmpd['CalYear'].unique()

In [285]:
conn = connect(':memory:')
# create tables
shooting.to_sql("shooting", conn)
lmpd.to_sql("lmpd", conn)
# convenient function
def sql(a_string):
    return(pd.read_sql(a_string, conn))

In [286]:
# sql('''
#     SELECT shooting.*, lmpd.*
#     FROM shooting
#     LEFT JOIN lmpd ON strftime('%Y', shooting.year) = lmpd.CalYear
#                     AND shooting.Name = lmpd.Employee_Name
# ''')

In [287]:
df = sql('''
    SELECT shooting.*, lmpd.*
    FROM shooting
    LEFT JOIN lmpd ON shooting.year = lmpd.CalYear
                    AND shooting.Name = lmpd.Employee_Name
''')

df.to_csv('data/shooting_join.csv')

In [288]:
# sql('''
#     SELECT
#         shooting.year AS Year,
#         shooting.Name,
#         shooting.Race,
#         shooting.Sex,
#         shooting.Age,
#         shooting.lethal,
#         lmpd.Hr_Worked,
#         lmpd.CalYear,
#         lmpd.Employee_Name
#     FROM shooting
#     LEFT JOIN lmpd ON strftime('%Y', shooting.year) = lmpd.CalYear
#                     AND shooting.Name = lmpd.Employee_Name
# ''')

In [289]:
sql('''
    SELECT
        shooting.year AS Year,
        shooting.Name,
        shooting.Race,
        shooting.Sex,
        shooting.Age,
        shooting.lethal,
        lmpd.Hr_Worked,
        lmpd.CalYear,
        lmpd.Employee_Name
    FROM shooting
    LEFT JOIN lmpd ON shooting.year = lmpd.CalYear
                    AND shooting.Name = lmpd.Employee_Name
''')


,Year,Name,Race,Sex,Age,lethal,Hr_Worked,CalYear,Employee_Name
0,2021,Reymart Relos,H,M,30,N,40.120014,2021.0,Reymart Relos
1,2021,Jonathan Haywood,B,M,45,N,41.348507,2021.0,Jonathan Haywood
2,2021,Michael Crowe,W,M,28,N,40.302582,2021.0,Michael Crowe
3,2021,Michael Conner,W,M,37,N,40.848659,2021.0,Michael Conner
4,2021,Galen Hinshaw,W,M,32,n,41.267821,2021.0,Galen Hinshaw
...,...,...,...,...,...,...,...,...,...
76,2018,Joseph Fox,W,M,26,Y,41.687500,2018.0,Joseph Fox
77,2018,Brendan Kaiser,W,M,23,Y,41.460922,2018.0,Brendan Kaiser
78,2018,Brandon Hogan,W,M,32,Y,44.089143,2018.0,Brandon Hogan
79,2018,William Mattingly,W,M,55,Y,42.867858,2018.0,William Mattingly


In [290]:
join = pd.read_csv("data/shooting_join.csv")
join.head()

,Unnamed: 0.1,index,year,Name,Race,Sex,Age,Ethnicity,Years of Service,lethal,...,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Hr_Worked
0,0,0,2021,Reymart Relos,H,M,30,U,4 months,N,...,51022.4,49231.92,221.57,4039.68,0.0,53493.17,18768.0,23.669192,35.503788,40.120014
1,1,1,2021,Jonathan Haywood,B,M,45,U,7,N,...,60840.0,63416.32,3206.90,4186.56,0.0,70809.78,16428.0,30.488615,45.732923,41.348507
2,2,2,2021,Michael Crowe,W,M,28,U,3,N,...,58864.0,61493.07,697.75,6042.72,0.0,68233.54,16940.0,29.563976,44.345964,40.302582
3,3,3,2021,Michael Conner,W,M,37,U,9,N,...,61921.6,65617.90,2088.27,5864.76,65.0,73635.93,16390.0,31.547067,47.320601,40.848659
4,4,4,2021,Galen Hinshaw,W,M,32,U,6,n,...,59862.4,63094.27,2999.71,6228.48,195.0,72517.46,16895.0,30.333784,45.500675,41.267821


In [291]:
join = join.sort_values(by="Hr_Worked", ascending=False)
join.head()

,Unnamed: 0.1,index,year,Name,Race,Sex,Age,Ethnicity,Years of Service,lethal,...,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Hr_Worked
49,49,49,2019,Anthony Garcia,W,M,33,NaN,3,N,...,50585.6,49336.68,22685.58,9757.36,4140.0,85919.62,25046.0,23.719558,35.579337,52.261644
24,24,24,2020,Brett Hankison,W,M,44,U,17,Y,...,59633.6,30963.60,12350.34,4488.00,10434.4,58236.34,28610.0,14.886346,22.329519,50.636438
55,55,55,2019,Jerry Codey,W,M,54,U,20+,N,...,61963.2,62762.98,19793.16,10254.48,0.0,92810.62,21787.0,30.174510,45.261764,48.409696
45,45,45,2020,Timothy Huber,W,M,38,U,2,N,...,58843.2,54064.70,16693.26,10072.96,2675.0,83505.92,29571.0,25.992644,38.988966,48.233720
68,68,67,2018,Aaron Browning,W,M,30,None,9,N,...,54392.0,53470.40,13665.16,9777.44,120.0,77033.00,36575.0,25.706923,38.560385,46.815065


In [292]:
join = join.sort_values(by="Years of Service", ascending=False)
join.head(15)

,Unnamed: 0.1,index,year,Name,Race,Sex,Age,Ethnicity,Years of Service,lethal,...,Annual_Rate,Regular_Rate,Overtime_Rate,Incentive_Allowance,Other,YTD_Total,ObjectId,Hr_Rate,Ot_Rate,Hr_Worked
60,60,60,2019,Multiple Officers,W,M,Varies,NaN,Varies,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,80,79,2018,Dallas Constant,W,M,23,None,Under 1 year,Y,...,44387.2,39900.72,742.96,5517.60,660.00,46821.28,39829.0,19.183038,28.774558,40.496539
29,29,29,2020,Brian Evanoff,W,M,40,U,9,N,...,73902.4,68475.20,1099.03,9288.16,0.00,78862.39,29892.0,32.920769,49.381154,40.428001
3,3,3,2021,Michael Conner,W,M,37,U,9,N,...,61921.6,65617.90,2088.27,5864.76,65.00,73635.93,16390.0,31.547067,47.320601,40.848659
68,68,67,2018,Aaron Browning,W,M,30,None,9,N,...,54392.0,53470.40,13665.16,9777.44,120.00,77033.00,36575.0,25.706923,38.560385,46.815065
48,48,48,2019,Tyler Holland,W,M,31,U,8yrs. 4m,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2021,Jonathan Haywood,B,M,45,U,7,N,...,60840.0,63416.32,3206.90,4186.56,0.00,70809.78,16428.0,30.488615,45.732923,41.348507
26,26,26,2020,Nathaniel Lovewell,W,M,31,U,6,Y,...,60902.4,56144.80,9889.99,7945.20,1225.00,75204.99,30543.0,26.992692,40.489038,44.697373
6,6,6,2021,Bianca Chambers,B,F,37,U,6,n,...,60902.4,41839.10,6650.39,2923.44,2423.78,53836.71,17189.0,20.114952,30.172428,44.238708
4,4,4,2021,Galen Hinshaw,W,M,32,U,6,n,...,59862.4,63094.27,2999.71,6228.48,195.00,72517.46,16895.0,30.333784,45.500675,41.267821


In [293]:
sql('''
    SELECT
        shooting.year AS Year,
        shooting.Name,
        shooting.Race,
        shooting.Sex,
        shooting.Age,
        shooting.lethal,
        lmpd.Hr_Worked,
        lmpd.CalYear,
        lmpd.Employee_Name
    FROM shooting
    LEFT JOIN lmpd ON shooting.year = lmpd.CalYear
                    AND shooting.Name = lmpd.Employee_Name
''')

,Year,Name,Race,Sex,Age,lethal,Hr_Worked,CalYear,Employee_Name
0,2021,Reymart Relos,H,M,30,N,40.120014,2021.0,Reymart Relos
1,2021,Jonathan Haywood,B,M,45,N,41.348507,2021.0,Jonathan Haywood
2,2021,Michael Crowe,W,M,28,N,40.302582,2021.0,Michael Crowe
3,2021,Michael Conner,W,M,37,N,40.848659,2021.0,Michael Conner
4,2021,Galen Hinshaw,W,M,32,n,41.267821,2021.0,Galen Hinshaw
...,...,...,...,...,...,...,...,...,...
76,2018,Joseph Fox,W,M,26,Y,41.687500,2018.0,Joseph Fox
77,2018,Brendan Kaiser,W,M,23,Y,41.460922,2018.0,Brendan Kaiser
78,2018,Brandon Hogan,W,M,32,Y,44.089143,2018.0,Brandon Hogan
79,2018,William Mattingly,W,M,55,Y,42.867858,2018.0,William Mattingly


In [296]:
join['Name'].value_counts()

Shane Denton            2
William Mattingly       2
Timothy Huber           2
Robert Ross             2
John Moore              1
                       ..
Benjamin Dean           1
Anthony Garcia          1
Nathaniel Richardson    1
Joshua Weyer            1
Alexander               1
Name: Name, Length: 77, dtype: int64